# RegEx Adv.

Going over more advanced syntax for RegEx. The following article will explore these advanced techniques syntax and methods.

1. re is a package for making regular expressions easy, such as the re.I method. This method allows us to ignore case sensitivity. So something like SQL, Sql, or sql will all be counted as one SQL. 

2. Capture groups can help us extract different regular expressions that may vary say like different versions of software or kinds of software (Ex: SQL vs. NoSQL or Python 3 and Python 2 

3. Lookarounds: there are four type of lookarounds - Positive Lookahead and Negative Lookahead along with Positive Lookbehind and Negative Lookbehind

Syntax is as follows:

Positive Lookahead: zzz(?=abc) Matches zzz only when it is followed by abc
Negative Lookahead zzz(?!abc) Matches zzz only when it is not followed by abc

Positive Lookbehind (?<=abc)zzz Matches zzz only when it is preceded by abc 
NEgative Lookbehind (?<!abc)zzz Matches zzz only when it is not preceded by abc 


Cheat Sheet:

These tips can help you remember the syntax for lookarounds:

Inside the parentheses, the first character of a lookaround is always ?.
If the lookaround is a lookbehind, the next character will be <, which you can think of as an arrow head pointing behind the match.
The next character indicates whether the lookaround is positive (=) or negative (!).

4. Backreferences: A way to identify capture groups that may repeat. (\w)\1 so this is any two word characters repeating. The \1 is key here.  

5. re.sub function: This looks like re.sub(pattern, repl, string, flags=0) and when working with pandas use this Series.str.replace(pat, repl, flags=0)

6. Multiple Capture Groups and Naming: You can specify multiple capture groups by just using multiple single groups like so (XXX)(XXX) this will capture multiple regexs. In addition you can name any regex by doing this ?P<NAME> followed by your capture group this will name it and make it easier to put into a dataframe.....the names become the column names!

In [1]:
import pandas as pd
import re

hn = pd.read_csv("hacker_news.csv")
titles = hn['title']

In [4]:
titles.str.contains('sql',flags=re.I).sum()

108

In [ ]:
hn_sql = hn[hn['title'].str.contains(r"\w+SQL", flags=re.I)].copy()

pattern=r'(\w+sql)'
hn_sql['flavor']= hn_sql['title'].str.extract(pattern,flags=re.I).str.lower()

sql_pivot=hn_sql.pivot_table('num_comments','flavor')

In [15]:
pattern = r"[Pp]ython ([\d\.]+)"

py_versions = titles.str.extract(pattern)
# py_versions_freq = dict(py_versions.value_counts())

In [17]:
py_versions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20099 entries, 0 to 20098
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       23 non-null     object
dtypes: object(1)
memory usage: 157.1+ KB


In [18]:
def first_10_matches(pattern):
    """
    Return the first 10 story titles that match
    the provided regular expression
    """
    all_matches = titles[titles.str.contains(pattern)]
    first_10 = all_matches.head(10)
    return first_10

first_10_matches(r"\b[Cc]\b")

13                 Custom Deleters for C++ Smart Pointers
220                        Lisp, C++: Sadness in my heart
221                  MemSQL (YC W11) Raises $36M Series C
353     VW C.E.O. Personally Apologized to President O...
365                      The new C standards are worth it
444           Moz raises $10m Series C from Foundry Group
508     BDE 3.0 (Bloomberg's core C++ library): Open S...
521          Fuchsia: Micro kernel written in C by Google
549     How to Become a C.E.O.? The Quickest Path Is a...
1282    A lightweight C++ signals and slots implementa...
Name: title, dtype: object

In [19]:
pattern = r"\b[Cc]\b[^.+]"
first_ten = first_10_matches(pattern)

In [ ]:
pattern= r'(?<!Series\s)\b[Cc]\b(?![\.\+])'
# Match instances of C or c where they are not preceded or followed by another word character.
# Exclude instances where the match is followed by a . or + character, without removing instances where the match occurs at the end of the string.
# Exclude instances where the word 'Sberies' immediately precedes the match.
c_mentions= titles.str.contains(pattern).sum()

In [ ]:
pattern=r'\b(\w+)\s\1\b'
# Write a regular expression to match cases of repeated words:
# We'll define a word as a series of one or more word characters that are preceded and followed by a boundary anchor.
# We'll define repeated words as the same word repeated twice, separated by a whitespace character.
repeated_words = titles[titles.str.contains(pattern)]

In [ ]:
email_variations = pd.Series(['email', 'Email', 'e Mail',
                        'e mail', 'E-mail', 'e-mail',
                        'eMail', 'E-Mail', 'EMAIL'])

email_uniform= pattern = r"e[\-\s]?mail"
email_uniform = email_variations.str.replace(pattern, "email", flags=re.I)
titles_clean = titles.str.replace(pattern, "email", flags=re.I)

In [ ]:
test_urls = pd.Series([
 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429',
 'http://www.interactivedynamicvideo.com/',
 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0',
 'http://evonomics.com/advertising-cannot-maintain-internet-heres-solution/',
 'HTTPS://github.com/keppel/pinn',
 'Http://phys.org/news/2015-09-scale-solar-youve.html',
 'https://iot.seeed.cc',
 'http://www.bfilipek.com/2016/04/custom-deleters-for-c-smart-pointers.html',
 'http://beta.crowdfireapp.com/?beta=agnipath',
 'https://www.valid.ly?param',
 'http://css-cursor.techstream.org'
])

pattern = r"https?://([\w\-\.]+)"

test_urls_clean = test_urls.str.extract(pattern, flags=re.I)
domains = hn['url'].str.extract(pattern, flags=re.I)
top_domains = domains.value_counts().head(5)

In [ ]:
pattern=r'(https?)://([\w\.\-]+)/?(.*)'
# The first capture group should include the protocol text, up to but not including ://.
# The second group should contain the domain, from after :// up to but not including /.
# The third group should contain the page path, from after / to the end of the string.
test_url_parts = test_urls.str.extract(pattern, flags=re.I)
url_parts = hn['url'].str.extract(pattern, flags=re.I)

In [ ]:
pattern = r"(?P<protocol>https?)://(?P<domain>[\w\.\-]+)/?(?P<path>.*)"

url_parts= hn['url'].str.extract(pattern,flags=re.I)